In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
service_credential = dbutils.secrets.get("databricks-demo-scope", "databricksSpValue")

spark.conf.set("fs.azure.account.auth.type.strgacdbricksasgmntwk18.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.strgacdbricksasgmntwk18.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.strgacdbricksasgmntwk18.dfs.core.windows.net", "3dee1d7c-419f-41c4-9f38-9d9e4691d813")
spark.conf.set("fs.azure.account.oauth2.client.secret.strgacdbricksasgmntwk18.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.strgacdbricksasgmntwk18.dfs.core.windows.net", "https://login.microsoftonline.com/8292855f-d9cf-4a1d-8eab-b6d80b03cc91/oauth2/token")

In [0]:
orders_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://week15inputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/order_data.csv")

In [0]:
orders_df.printSchema()

In [0]:
orders_df.write.partitionBy("Country").mode("overwrite").format("parquet").save("abfss://week15outputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/parquet/orders.parquet")

In [0]:
orders_df.write.partitionBy("Country").mode("overwrite").format("delta").save("abfss://week15outputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/delta/orders.delta")

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS retail_db;
USE retail_db;
CREATE TABLE IF NOT EXISTS retail_db.orders_parqeut USING PARQUET LOCATION 'abfss://week15outputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/parquet/orders.parquet';
CREATE TABLE IF NOT EXISTS retail_db.orders_delta USING DELTA LOCATION 'abfss://week15outputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/delta/orders.delta';


In [0]:
%sql
describe history orders_delta;

In [0]:
orders_df.write. \
    partitionBy("Country"). \
        mode("overwrite"). \
            format("delta"). \
                option("path", "abfss://week15outputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/delta/orders_new.delta"). \
                    saveAsTable("orders_new")

In [0]:
%sql
INSERT INTO retail_db.orders_new VALUES ('11223344', '22343', 'Cat Food', 24, '30-03-2025 22:20', 45.8, 123456, 'India');

In [0]:
orders_append = spark.read.format("csv").option("inferSchema", True).option("header", True).load("abfss://week15inputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/order_data_append.csv").withColumn("InvoiceNo", col("InvoiceNo").cast("string")).withColumn("StockCode", col("StockCode").cast("string"))

In [0]:
orders_append.write.partitionBy("Country").mode("append").format("delta").save("abfss://week15outputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/delta/orders_new.delta")

In [0]:
%sql
describe history retail_db.orders_new;

In [0]:
%sql
COPY INTO retail_db.orders_new
FROM(
  SELECT
  cast(invoiceno as string) as invoiceno,
  cast(stockcode as string) as stockcode,
  description,
  cast(Quantity as integer) as quantity,
  invoicedate,
  cast(unitprice as double),
  cast(customerId as integer),
  country
  FROM "abfss://week15inputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/order_data_copy.csv"
)

FILEFORMAT = CSV
FORMAT_OPTIONS ('header' ='true');

In [0]:
%sql
SELECT * FROM retail_db.orders_new limit 1;

In [0]:
%sql
describe history retail_db.orders_new;

In [0]:
orders_new_schema_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://week15inputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/order_data_append_new_schema.csv").withColumn("InvoiceNo", col("InvoiceNo").cast("string")).withColumn("StockCode", col("StockCode").cast("string"))

In [0]:
orders_new_schema_df.write.partitionBy("Country").mode("append").format("delta").option("mergeSchema", True). \
    save("abfss://week15outputdatasets@strgacdbricksasgmntwk18.dfs.core.windows.net/delta/orders_new.delta")

In [0]:
%sql
select * from retail_db.orders_new where state is not null;

In [0]:
%sql
select * from retail_db.orders_delta limit 10;

In [0]:
%sql
select * from orders_delta where CustomerID = 12431 and Country = 'Australia';

In [0]:
%sql
UPDATE retail_db.orders_delta SET Country = 'India' WHERE customerid = 12431 and country = 'Australia';

In [0]:
%sql
DELETE FROM orders_delta where country = 'Brazil' and quantity = 16;

In [0]:
%sql
describe history orders_delta;

In [0]:
%sql
RESTORE retail_db.orders_delta TO VERSION AS OF 0;

In [0]:
%sql
describe history orders_delta;

In [0]:
%sql
SELECT * FROM orders_delta where country = 'Brazil' and quantity = 16;